# Module9_2: Variant Calling

Grab an interactive node for the class 

`qsub -I -l nodes=1:ppn=8 -l walltime=3:00:00 -q hotel`

**TIP** if you start overwriting your own line, use:

`eval $(resize)`

## Create a results day 2 folder in your scratch directory and move to this directory
```
mkdir -p ~/scratch/Module_9/results_day2
cd ~/scratch/Module_9/results_day2
```

---
## Generate pileup (without ref)
```
samtools mpileup -Q 0 ~/biom262_2019/Module_9/materials/CPTRES7.realigned.chr21.bam > CPTRES7.chr21.pileup

samtools mpileup -Q 0 ~/biom262_2019/Module_9/materials/CPTRES4.realigned.chr21.bam > CPTRES4.chr21.pileup
```

---

## Call variants with varscan (need interactive node for RAM needs)
```
varscan somatic CPTRES4.chr21.pileup CPTRES7.chr21.pileup CPTRES7vs4 --output-vcf
```

---
## Calling CNV

CNV calling with CNVkit batch

Currently non-functional

```
cnvkit.py batch \
~/biom262_2019/Module_9/materials/CPTRES7.chr21.bam \
--normal ~/biom262_2019/Module_9/materials/CPTRES4.chr21.bam \
--targets ~/biom262_2019/Module_9/materials/Sureselect.chr21.bed \
--annotate ~/biom262_2019/Module_9/resources/refFlat.txt \
--fasta ~/biom262_2019/Module_9/resources/chr21.fa.gz \
--output-reference myref.cnn --output-dir .  \
--count-reads --diagram --scatter
```

---
## VCF file manipulation

Let's merge the snp and indel files 

zip (bgzip) and index the vcf file

```
for file in CPTRES7vs4*vcf; do bgzip $file; tabix -p vcf $file.gz; done
```

Concatenate the snp and indel files
```
bcftools concat -O z -o CPTRES7vs4.vcf.gz CPTRES7vs4.snp.vcf.gz CPTRES7vs4.indel.vcf.gz
```

Flag variants with coverage less than 20
```
bcftools filter -s "DP20" -O z -e 'INFO/DP<20' CPTRES7vs4.vcf.gz > CPTRES7vs4.filter.vcf.gz
```

keep PASS and germline
```
bcftools filter -O z -i 'FILTER=="PASS" & INFO/SS=="1"' CPTRES7vs4.filter.vcf.gz > CPTRES7vs4.germ.vcf.gz
```

What is the transition to transversion ratio?
```
bcftools stats CPTRES7vs4.germ.vcf.gz
```

---
## Variant Annotation

For this part we will use the results of cphort germien sequencing (GATK HapotypeCaller). Note: genotype have been shuffled to preserve privacy. Note2: GATK was run on individual file, not cohort, hence no use of gVCF : missing are assumed homozygous reference. 

Breakmulti allele sites

```
vcfbreakmulti ~/biom262_2019/Module_9/materials/GMTS_all.shuf.vcf.gz | bgzip -c >  GMTS_all.shuf.BM.vcf.gz

tabix -p vcf GMTS_all.shuf.BM.vcf.gz
```

run Table annovar on the GMTS variant file. 
```
~/biom262_2019/Module_9/resources/annovar/table_annovar.pl --vcfinput --nastring . --protocol refGene --operation g --buildver hg19 --outfile GMTSann GMTS_all.shuf.BM.vcf.gz ~/biom262_2019/Module_9/resources/annovar/humandb/
```

export genotypes to TSV file. One sample.variant per row. 
```
vcf2tsv -g GMTS_all.shuf.BM.vcf.gz > GMTS.geno.txt
```